In [94]:
import numpy as np
import pandas as pd
from scipy import log,exp,sqrt,stats
import math
import time

## Option pricing with Black-Scholes model

In [140]:
def BlackScholes_call(S,E,T,t,rf,sigma):
    #first we have to calculate d1 and d2 parameters
    d1 = (log(S/E)+(rf+(sigma**2)/2)*(T-t))/(sigma*sqrt(T-t))
    d2 = d1-sigma*sqrt(T-t)
    
    #we need N(x) normal distribution function
    return S*stats.norm.cdf(d1)-E*exp(-rf*(T-t))*stats.norm.cdf(d2)

In [141]:
def BlackScholes_put(S,E,T,t,rf,sigma):
    #first we have to calculate d1 and d2 parameters
    d1 = (log(S/E)+(rf+sigma**2/2)*(T-t))/(sigma*sqrt(T-t))
    d2 = d1-sigma*sqrt(T-t)
    
    #we need N(x) normal distribution function
    return -S*stats.norm.cdf(-d1)+E*exp(-rf*(T-t))*stats.norm.cdf(-d2)

In [161]:
S0 = 100 #underlying stock price at t = 0
E = 100 #strike price
T = 1 #expiry 1 = 1 year = 365 days
t = 0
rf = 0.05 #risk-free rate, 5%
sigma = 0.2 #volatility of underlying stock, 20%

In [162]:
print("Call option price according to Black-Scholes model:", BlackScholes_call(S0, E, T, t, rf, sigma))
print("Put option price according to Black-Scholes model:", BlackScholes_put(S0, E, T, t, rf, sigma))

Call option price according to Black-Scholes model: 10.450583572185565
Put option price according to Black-Scholes model: 5.573526022256971


## Option pricing with Monte-Carlo Simulation

In [171]:
class OptionPricing:
    t = 0.5 #to use this parameter uncomment line 22 & 38
    
    def __init__(self,S0,E,T,rf,sigma,iterations):
        self.S0 = S0
        self.E = E
        self.T = T
        self.rf = rf
        self.sigma = sigma
        self.iterations = iterations #how many stock prices we want to simulate
        
    def call_option_simulation(self):
        #payoff function is max(0,S-E) for call option
        #We have 2 columns: first column with 0s, second column will store the payoff S-E
        option_data = np.zeros([self.iterations, 2]) #matrice de 0 de dim [iterations x 2]
        
        #dimensions: 1 dimensional array with as many items as the iterations
        rand = np.random.normal(0, 1, [1,self.iterations])
        
        #equation for the S(t) stock price (which we need for payoff function S-E)
        stock_price = self.S0*np.exp(self.T*(self.rf - (self.sigma**2/2))+self.sigma*np.sqrt(self.T)*rand)
        #stock_price = self.S0*np.exp((self.T-t)*(self.rf - (self.sigma**2/2))+self.sigma*np.sqrt(self.T-t)*rand)
        
        #we need S-E because we have to calculate the max(S-E,0)
        option_data[:,1] = stock_price - self.E 
        
        #average for the Monte-Carlo method
        #np.amax() returns the max(0,S-E) according to the formula
        average = np.sum(np.amax(option_data, axis=1))/self.iterations
        
        #have to use the exp(-rT) discount factor (future payoff)
        return np.exp(-self.rf*self.T)*average
    
    def put_option_simulation(self):
        option_data = np.zeros([self.iterations, 2])
        rand = np.random.normal(0, 1, [1,self.iterations])
        stock_price = self.S0*np.exp(self.T*(self.rf - (self.sigma**2/2))+self.sigma*np.sqrt(self.T)*rand)
        #stock_price = self.S0*np.exp((self.T-t)*(self.rf - (self.sigma**2/2))+self.sigma*np.sqrt(self.T-t)*rand)
        #we need E-S because put option
        option_data[:,1] = self.E - stock_price
        average = np.sum(np.amax(option_data, axis=1))/self.iterations
        return np.exp(-self.rf*self.T)*average
    
#On a une formule pour calculer le prix S(T) du stock a la maturite
#Donc on genere pleins de prix differents, on fait la soustraction ensuite avec le strike price/expiry price
#Puis on prend tous les payoffs obtenus (combien nous rapporte l'exercice de l'option pour chaque prix generes)
#Et on prend la moyenne des payoffs qui nous donne un prix qui correspond donc au prix de l'option

In [172]:
iterations = 10000000
model = OptionPricing(S0, E, T, rf, sigma, iterations)

print("Call option price according to Black-Scholes model:", model.call_option_simulation())
print("Put option price according to Black-Scholes model:", model.put_option_simulation())

Call option price according to Black-Scholes model: 10.443027262746199
Put option price according to Black-Scholes model: 5.571936011989728
